In [1]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
import webbrowser
import markdown


In [2]:
from dotenv import load_dotenv
load_dotenv()  ###load openai key
# Set up API keys
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [3]:
class BlogCrewAI:
    model = "gpt-4o-mini"  # You can change this to your preferred model
    def __init__(self):
        # Initialize tools
        self.search_tool = SerperDevTool()
        
        # Create agents
        self.analyst = Agent(
            role='Content Research Analyst',
            goal='Conduct thorough analysis of topics and identify key insights',
            backstory="""You are an expert content analyst with years of experience in 
            researching and analyzing various topics. You have a keen eye for identifying 
            trends, patterns, and valuable insights that make content engaging and informative.""",
            tools=[self.search_tool],
            verbose=True
        )
        
        self.writer = Agent(
            role='Blog Content Writer',
            goal='Create engaging and well-researched blog posts',
            backstory="""You are a skilled content writer with expertise in creating 
            compelling blog posts. You excel at turning complex information into 
            readable, engaging content that resonates with readers while maintaining 
            accuracy and providing value.""",
            verbose=True
        )

    def create_blog(self, topic: str):
        # Analysis task
        analysis_task = Task(
            description=f"""Thoroughly analyze the topic: {topic}
            1. Research current trends and developments
            2. Identify key points and insights
            3. Find relevant statistics and examples
            4. Analyze the target audience's interests
            5. Determine the best angle for the blog post
            
            Provide a detailed analysis report including:
            - Main points to cover
            - Supporting data and examples
            - Recommended content structure
            - Key takeaways for readers""",
            agent=self.analyst,
            expected_output="A comprehensive analysis report with structured insights and recommendations",
            tools=[self.search_tool]
        )

        # Writing task
        writing_task = Task(
            description=f"""Using the analysis provided, create an engaging blog post about {topic}.
            The blog post should:
            1. Have an attention-grabbing introduction
            2. Include relevant data and examples
            3. Be well-structured with clear sections
            4. Maintain a conversational yet professional tone
            5. End with a compelling conclusion
            
            Format the blog post in markdown with:
            - Clear headings and subheadings
            - Properly formatted lists and quotes
            - A compelling title
            - Meta description for SEO""",
            agent=self.writer,
            expected_output="A complete, well-formatted blog post in markdown format",
            context=[analysis_task]
        )

        # Create and run the crew
        crew = Crew(
            agents=[self.analyst, self.writer],
            tasks=[analysis_task, writing_task],
            process=Process.sequential,
            verbose=True
        )

        # Execute the crew's tasks and return the result
        result = crew.kickoff()
        return result


In [4]:
def main():
    # Initialize the BlogCrewAI
    blog_crew = BlogCrewAI()
    
    # Get topic from user
    topic = input("Enter the blog topic: ")
    
    # Generate blog
    print("\nGenerating blog post...\n")
    result = blog_crew.create_blog(topic)
    
    print("\nBlog post generated successfully!\n")
    
    # Extract the content from CrewOutput object
    content = str(result)  # Convert CrewOutput to string
    
    # Convert markdown to HTML
    html_body = markdown.markdown(content)
    
    # Write the result to an HTML file with styling
    html_content = f"""
    <html>
    <head>
        <title>Blog Post: {topic}</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                line-height: 1.6;
                max-width: 800px;
                margin: 0 auto;
                padding: 20px;
            }}
            h1, h2, h3 {{
                color: #333;
            }}
            p {{
                margin-bottom: 1.2em;
            }}
            code {{
                background-color: #f4f4f4;
                padding: 2px 5px;
                border-radius: 3px;
            }}
        </style>
    </head>
    <body>
        <h1>{topic}</h1>
        <div>{html_body}</div>
    </body>
    </html>
    """
    
    # Specify the output HTML file path
    output_file = "blog_post.html"
    
    # Write the HTML content to the file
    with open(output_file, "w") as file:
        file.write(html_content)
    
    # Open the HTML file in the default web browser
    webbrowser.open(f'file://{os.path.realpath(output_file)}')


In [5]:
if __name__ == "__main__":
    main()


Generating blog post...

# Agent: Content Research Analyst
## Task: Thoroughly analyze the topic: agi intelligence
            1. Research current trends and developments
            2. Identify key points and insights
            3. Find relevant statistics and examples
            4. Analyze the target audience's interests
            5. Determine the best angle for the blog post
            
            Provide a detailed analysis report including:
            - Main points to cover
            - Supporting data and examples
            - Recommended content structure
            - Key takeaways for readers


# Agent: Content Research Analyst
## Thought: To conduct a thorough analysis of agi intelligence, I need to gather the most up-to-date information regarding current trends, developments, key points, and relevant statistics. This will help me understand the audience's interests and determine the best angle for a blog post.
## Using tool: Search the internet
## Tool Input: 
"{\"